In [208]:
import os


In [209]:
file = open('Dane/Gr 1/fe3o4_ref.hkl','r')
read_content = file.read()
print(read_content)
file.close()

 Pattern#  1 Phase No.:  1 Fe3O4               90 reflections, N&T:        0    0.00    (The # of eff. reflections may be lower)
!  SPGR: 227:1;  CELL:      8.39767     8.39767     8.39767    90.00000    90.00000    90.00000
   h   k   l    Mult  sinT/lamb    2T       Fwhm         F2      sf2
   1   1   1       8   0.10313   18.2835   0.20478    3244.257     417.775
   0   2   2      12   0.16841   30.0743   0.24475   25969.875     569.693
   1   1   3      24   0.19747   35.4233   0.26460   67434.758     572.174
   2   2   2       8   0.20625   37.0544   0.27089   17647.762     489.592
   0   0   4       6   0.23816   43.0505   0.29502   87436.758    2255.385
   1   3   3      24   0.25953   47.1354   0.31244    1262.890     265.764
   4   2   2      24   0.29169   53.4069   0.34090   16192.868     700.160
   1   1   5      24   0.30938   56.9313   0.35789   46030.844     839.166
   3   3   3       8   0.30938   56.9313   0.35789   41321.898     753.319
   4   4   0      12   0.33681 

In [210]:
def files_names(path):
    with open("names.txt", "w", encoding="utf-8") as names_f:
#names_f = open('files_names.txt','r+')
        for root, dirs, files in os.walk(path):
            names_f.write(f"[PATH]{root}\n")
            for d in dirs: 
                names_f.write(f"[DIR]{d}\n")
            for f in files: 
                names_f.write(f"[FILE]{f}\n")
            names_f.write("\n")
    with open("names.txt", "r", encoding="utf-8") as names_f:
        n = names_f.read()
        print(n)
path = r"D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane"
files_names(path)

[PATH]D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane
[DIR]Gr 1
[DIR]Gr 2
[DIR]Gr 3
[DIR]Gr01
[DIR]Gr02
[DIR]Gr02 (1)
[DIR]Gr03
[DIR]Gr03 (1)
[DIR]Gr04
[DIR]Gr05
[DIR]Gr06
[DIR]Grupa 1
[DIR]Grupa 2
[DIR]Grupa 3
[DIR]Grupa 4
[DIR]Grupa 5
[DIR]Grupa01
[DIR]Grupa02
[DIR]Grupa03
[DIR]Grupa04
[DIR]Grupa05
[DIR]NIST660

[PATH]D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane\Gr 1
[DIR]testy
[FILE]aaaa.BGR
[FILE]Fe2O3.cel
[FILE]Fe3O4.cel
[FILE]fe3o4_ref.bac
[FILE]fe3o4_ref.hkl
[FILE]Fe3O4_ref.new
[FILE]fe3o4_ref.out
[FILE]fe3o4_ref.pcr
[FILE]fe3o4_ref.prf
[FILE]fe3o4_ref.sum
[FILE]fe3o4_ref.sym
[FILE]Fe3O4_ref.xrdml
[FILE]fe3o4_ref1.fst
[FILE]m3_fe3o4.bac
[FILE]m3_fe3o4.hkl
[FILE]M3_Fe3O4.new
[FILE]m3_fe3o4.out
[FILE]m3_fe3o4.pcr
[FILE]m3_fe3o4.prf
[FILE]m3_fe3o4.sum
[FILE]m3_fe3o4.sym
[FILE]M3_Fe3O4.xrdml
[FILE]m3_fe3o41.fst
[FILE]MagnetiteStructure.PNG
[FILE]Mielona_M3.PNG
[FILE]Otoczenia.PNG
[FILE]Prezentacja1.pptx
[FILE]Referencja.PNG
[FILE]Sample_S.xrdml
[FILE]Sample_S_ZnO.xrdml

[PATH]D

In [211]:
def document_new():
    new = open('new_files.txt','w')
    with open('names.txt','r') as names_f:
        for line in names_f:
            if '[PATH]' in line:
                new.write(f"{line}")
            elif '[FILE]' and '.new' in line:
                    new.write(f"{line}")
        new.write(f"\n")

document_new()
with open('new_files.txt','r') as new_doc:
    new = new_doc.read().replace("\\","/")
   # new = new.replace("[FILE]","")
with open("new_files.txt", "w", encoding="utf-8") as new_doc:
    new_doc.write(new) 
 

In [212]:
def read_inside(path):
    file = open(str(path),'r')
    read_content = file.read()
    print(read_content)
    file.close()

In [ ]:
def read_new_doc():
    path = []
    with open('new_files.txt','r') as new_doc:
        lines = new_doc.readlines()
        for i,line in enumerate(lines):
            if '[PATH]' in line:
                clean_path = line.replace("[PATH]","").strip()
                n = 1
                if i+n< len(lines) and '[FILE]' in lines[i+n]:
                    while '[FILE]' in lines[i+n]:
                        # if i+n < len(lines) and '[FILE]' in lines[i+n]:
                        clean_file = lines[i+n].replace("[FILE]","/").strip()
                        full_path = clean_path+clean_file
                        path.append(full_path)
                        n=n+1
        print(path)
    return path
all_paths = read_new_doc()             

['D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane/Gr 1/Fe3O4_ref.new', 'D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane/Gr 1/M3_Fe3O4.new', 'D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane/Gr 1/testy/Fe3O4_50min_800rpm.new', 'D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane/Gr 1/testy/Fe3O4_ref.new', 'D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane/Gr 1/testy/M3_Fe3O4.new', 'D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane/Gr 2/Fe3O4_100micro.new', 'D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane/Gr 2/Fe3O4_800RPM_50min_3mmZrO2.new', 'D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane/Gr 3/Fe3O4_50min_800rpm.new', 'D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane/Gr 3/Fe3O4_ref.new', 'D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane/Gr01/FullProf_input.new', 'D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane/Gr01/WO3_600RPM_10min.new', 'D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane/Gr01/wo3_600rpm_120min.new', 'D:/nai/Nati/studia/inzynierka/github/XRD-AI/Dane/Gr01/WO3_600RPM_20min

In [217]:
# for i in range(len(all_paths)):
read_inside(all_paths[15])
    #print(all_paths[i])

ZrO2 monoclinic
! Current global Chi2 (Bragg contrib.) =      3.143    
NPATT      1       1 <- Flags for patterns (1:refined, 0: excluded)
W_PAT   1.000
!Nph Dum Ias Nre Cry Opt Aut
   1   0   0   0   0   0   1
!Job Npr Nba Nex Nsc Nor Iwg Ilo Res Ste Uni Cor Anm Int
   0   7   0   1   0   1   0   0   0   0   0   0   0   0  !-> Patt#: 1
!
!File names of data(patterns) files
zro2_raw.xrdml
!
!Mat Pcr NLI Rpa Sym Sho
   1   2   0   0   1   0
!Ipr Ppl Ioc Ls1 Ls2 Ls3 Prf Ins Hkl Fou Ana
   0   2   0   0   4   0   3  13   2   0   0  !-> Patt#: 1
!
! Lambda1  Lambda2    Ratio    Bkpos    Wdt    Cthm     muR   AsyLim   Rpolarz  2nd-muR -> Patt# 1
 1.540600 1.544390  0.50000   90.000 25.0000  0.9000  0.0000   90.00    0.0000  0.0000
!
!NCY  Eps  R_at  R_an  R_pr  R_gl
 20  0.30  0.90  0.90  0.90  0.90
!     Thmin       Step       Thmax    PSD    Sent0  -> Patt#: 1
    10.0299   0.033423    79.9833   0.000   0.000
!
! Excluded regions (LowT  HighT) for Pattern#  1
        0.00       15.00
! 
